In [1]:
cd ..

/Users/arthur/Documents/Flatiron/phase_4/Project/pneumonia_CT_scan


In [2]:
import os, shutil, random
import pandas as pd
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
train_dir = 'data/split/train'
val_dir = 'data/split/val'
test_dir = 'data/split/test'

train_pneu = train_dir + '/PNEUMONIA'
val_pneu = val_dir + '/PNEUMONIA'
test_pneu = test_dir + '/PNEUMONIA'
train_non_pneu = train_dir + '/NORMAL'
val_non_pneu = val_dir + '/NORMAL'
test_non_pneu = test_dir + '/NORMAL'

In [4]:
print('There are', len(os.listdir(train_pneu)), 'pneumonia images in the training set')
print('There are', len(os.listdir(val_pneu)), 'pneumonia images in the validation set')
print('There are', len(os.listdir(test_pneu)), 'pneumonia images in the testing set')
print('There are', len(os.listdir(train_non_pneu)), 'non-pneumonia images in the training set')
print('There are', len(os.listdir(val_non_pneu)), 'non-pneumonia images in the validation set')
print('There are', len(os.listdir(test_non_pneu)), 'non-pneumonia images in the testing set')

There are 2564 pneumonia images in the training set
There are 855 pneumonia images in the validation set
There are 854 pneumonia images in the testing set
There are 950 non-pneumonia images in the training set
There are 317 non-pneumonia images in the validation set
There are 316 non-pneumonia images in the testing set


In [5]:
# Get all the data in the directory data/train (2564+950 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_dir, 
        target_size=(64, 64), batch_size=2564+950)

# Get all the data in the directory data/val (855+317 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_dir, 
        target_size=(64, 64), batch_size=855+317)

# Get all the data in the directory data/test (854+316 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir, 
        target_size=(64, 64), batch_size=854+316)

# Create the datasets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)

Found 3514 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
Found 1170 images belonging to 2 classes.


In [6]:
print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(val_images))
print(np.shape(val_labels))
print(np.shape(test_images))
print(np.shape(test_labels))

(3514, 64, 64, 3)
(3514, 2)
(1172, 64, 64, 3)
(1172, 2)
(1170, 64, 64, 3)
(1170, 2)


In [7]:
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

{'NORMAL': 0, 'PNEUMONIA': 1}
{'NORMAL': 0, 'PNEUMONIA': 1}
{'NORMAL': 0, 'PNEUMONIA': 1}


In [8]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)
print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(3514, 12288)
(1170, 12288)
(1172, 12288)


In [9]:
train_y = np.reshape(train_labels[:,0], (train_images.shape[0],1))
test_y = np.reshape(test_labels[:,0], (test_images.shape[0],1))
val_y = np.reshape(val_labels[:,0], (val_images.shape[0],1))

## Dummy Model

In [23]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(train_images, train_labels)

DummyClassifier(strategy='most_frequent')

In [25]:
dummy_clf.score(train_images,train_labels)

0.7296528173022196

In [26]:
dummy_clf.score(val_images,val_labels)

0.7295221843003413

In [27]:
dummy_clf.score(test_images,test_labels)

0.7299145299145299

## Model 1: Neural Network

In [10]:
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,)))
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                     train_y,
                     epochs=50,
                     batch_size=32,
                     validation_data=(val_img, val_y))

Epoch 1/50
110/110 [==============================] - 1s 9ms/step - loss: 0.6426 - accuracy: 0.7020 - val_loss: 0.4997 - val_accuracy: 0.7304
Epoch 2/50
110/110 [==============================] - 0s 2ms/step - loss: 0.4436 - accuracy: 0.8027 - val_loss: 0.3741 - val_accuracy: 0.8063
Epoch 3/50
110/110 [==============================] - 0s 2ms/step - loss: 0.3423 - accuracy: 0.8524 - val_loss: 0.2824 - val_accuracy: 0.8857
Epoch 4/50
110/110 [==============================] - 0s 2ms/step - loss: 0.2955 - accuracy: 0.8715 - val_loss: 0.2241 - val_accuracy: 0.9181
Epoch 5/50
110/110 [==============================] - 0s 2ms/step - loss: 0.2543 - accuracy: 0.8947 - val_loss: 0.2570 - val_accuracy: 0.9010
Epoch 6/50
110/110 [==============================] - 0s 2ms/step - loss: 0.2306 - accuracy: 0.9072 - val_loss: 0.2788 - val_accuracy: 0.8780
Epoch 7/50
110/110 [==============================] - 0s 2ms/step - loss: 0.2638 - accuracy: 0.8885 - val_loss: 0.2018 - val_accuracy: 0.9266
Epoch 

In [12]:
results_train = model.evaluate(train_img, train_y)

110/110 [==============================] - 0s 2ms/step - loss: 0.1732 - accuracy: 0.9300


In [13]:
results_test = model.evaluate(test_img, test_y)

37/37 [==============================] - 0s 1ms/step - loss: 0.2371 - accuracy: 0.9077


In [14]:
results_train

[0.17318986356258392, 0.9299942851066589]

In [15]:
results_test

[0.2370913028717041, 0.9076923131942749]

## Model 2: CNN

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [22]:
history = model.fit(train_images,
                    train_y,
                    epochs=30,
                    batch_size=100,
                    validation_data=(val_images, val_y))

Epoch 1/30
36/36 [==============================] - 8s 211ms/step - loss: 0.1222 - acc: 0.9533 - val_loss: 0.1546 - val_acc: 0.9411
Epoch 2/30
36/36 [==============================] - 7s 207ms/step - loss: 0.1220 - acc: 0.9542 - val_loss: 0.2114 - val_acc: 0.9224
Epoch 3/30
36/36 [==============================] - 8s 229ms/step - loss: 0.1207 - acc: 0.9553 - val_loss: 0.1670 - val_acc: 0.9326
Epoch 4/30
36/36 [==============================] - 7s 199ms/step - loss: 0.1179 - acc: 0.9559 - val_loss: 0.1537 - val_acc: 0.9445
Epoch 5/30
36/36 [==============================] - 7s 191ms/step - loss: 0.1181 - acc: 0.9562 - val_loss: 0.1750 - val_acc: 0.9369
Epoch 6/30
36/36 [==============================] - 7s 192ms/step - loss: 0.1170 - acc: 0.9536 - val_loss: 0.1962 - val_acc: 0.9283
Epoch 7/30
36/36 [==============================] - 7s 192ms/step - loss: 0.1175 - acc: 0.9562 - val_loss: 0.1518 - val_acc: 0.9437
Epoch 8/30
36/36 [==============================] - 7s 208ms/step - loss: 0.

In [18]:
results_train = model.evaluate(train_images, train_y)

110/110 [==============================] - 2s 17ms/step - loss: 0.1570 - acc: 0.9388


In [19]:
results_test = model.evaluate(test_images, test_y)

37/37 [==============================] - 1s 17ms/step - loss: 0.1933 - acc: 0.9325


In [20]:
results_train

[0.156983882188797, 0.9388161897659302]

In [21]:
results_test

[0.1932610422372818, 0.9324786067008972]

## Overampling